<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-данных" data-toc-modified-id="Загрузка-библиотек-и-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка библиотек и данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Обоснование-решения" data-toc-modified-id="Обоснование-решения-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Обоснование решения</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

**Цель проекта:**
Защитить данные клиентов страховой компании. 
Для этого необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Необходимо также обосновать корректность работы метода. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка библиотек и данных

Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Импорт данных

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')
    
#вывод параметров датасета для проверки    
display(df.shape)
display(df.info())
display(df.describe())
display(df.head())


(5000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Подготовка данных

In [3]:
# Проверка на явные дубликаты
print('Количество явных дубликатов', df.duplicated().sum())

# Удаление явных дубликатов
df.drop_duplicates(inplace=True)

# сброс индексов
df=df.reset_index(drop=True)

# Проверка результата
print('Количество явных дубликатов после удаления', df.duplicated().sum())
print(df.shape)
print(df.info())
print(df.describe())

Количество явных дубликатов 153
Количество явных дубликатов после удаления 0
(4847, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB
None
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  4847.000000  4847.000000   4847.000000  4847.000000        4847.000000
mean      0.498453    31.023932  39895.811842     1.203425           0.152259
std       0.500049     8.487995   9972.953985     1.098664           0.468934
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24

Формирование матрицы признаков и вектора целевого признака (таргета)

In [4]:
X=df.drop(columns='Страховые выплаты').values
y=df['Страховые выплаты'].values

print('Тип данных признаков:',type(X))
print('Тип данных таргета:',type(y))
print('Размер данных признаков:',X.shape)
print('Размер данных таргета:',y.shape)

Тип данных признаков: <class 'numpy.ndarray'>
Тип данных таргета: <class 'numpy.ndarray'>
Размер данных признаков: (4847, 4)
Размер данных таргета: (4847,)


**Вывод:**
В процессе импорта и подготовки данных было выполнено следующее:
- первичный обзор данных
- устранение явных дубликатов
- разбиение данных на матрицу признаков и вектор целевого признака.

## Умножение матриц

### Обоснование решения

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $a$ - вектор предсказаний, который получается умножением матрицы признаков $X$ на вектор весов $w$

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии при умножении признаков на обратимую матрицу не меняется. При подобном подходе не происходит потери данных, следовательно его можно использовать для защиты информации.

**Обоснование:** 
1. Умножаем матрицу признаков $X$ на обратимую матрицу $A$: 

    Вектор предсказаний: 
$$
a = XAw
$$
    Берем формулу обучения и подставляем в вектор предсказаний: 
$$
w = (X^T X)^{-1} X^T y
$$

    Теперь веса линейной регрессии, согласно формуле обучения, выглядят так :

$$
w' = ((XA)^T XA)^{-1} (XA)^T y)
$$

2. Подставляем веса $w'$ в фомулу для $a$: 

$$
a = XA((XA)^T XA)^{-1} (XA)^T y)
$$

3. Транспонированное произведение матриц равно произведению транспонированных матриц: $(AB)^T = B^TA^T$  
    применив его, получаем:

$$
a = XA(A^TX^T XA)^{-1} A^TX^T y)
$$

4. Обратная матрица от произведения двух матриц А и B равна произведению обратной матрицы от B  на обратную матрицу от А: $(AB)^{-1} = B^{-1}A^{-1}$  
применив это, получаем:

$$
a = XA((X^T XA)^{-1}(A^T)^{-1}A^TX^T y)
$$

    преобразуем:

$$
a = XAA^{-1}((X^T X)^{-1}(A^T)^{-1}A^TX^T y)
$$

5. Умножение матрицы на обратную ей дает единичную матрицу: $AA^{-1}=E$  
воспользуемся и преобразуем опять:

$$
a = XE((X^T X)^{-1}(A^T)^{-1}A^TX^T y)
$$

6. Согласно предыдущему утверждению, также и $(A^T)^{-1}A^T=E$, тогда, преобразовав, получим:

$$
a = XE((X^T X)^{-1}EX^T y)
$$

7. Произведение любой матрицы на единичную матрицу не изменяет саму матрицу, поэтому просто убираем E из произведений в формуле и получаем:

$$
a = X(X^T X)^{-1}X^T y
$$

8. Применяем формулу обучения $w = (X^T X)^{-1} X^T y$ и получаем исходную формулу предсказания:

$$
a = X(X^T X)^{-1}X^T y=Xw
$$

Таким образом мы доказываем, что качество линейной регрессии при умножении признаков на обратимую матрицу не меняется.

## Алгоритм преобразования

**Алгоритм**
содержит следующие шаги:
1. Измеряем качество линейной регресси метрикой R2 на исходных данных.
2. Умножаем матрицу признаков на созданную нами обратимую матрицу и получаем новую матрицу признаков. 
3. Обучаем новую модель линейной регрессии на новой  матрице признаков и сравниваем её качество с исходной моделью, опять же по метрике R2.

Создаем функцию измерения качества линейной регрессии.

In [5]:
def measurement(X,y):
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    return r2_score(y, predictions)

Создаем функцию, возвращающую случайную матрицу с проверкой её обратимости

In [6]:
# определим функцию, возвращающую случайную матрицу с проверкой её обратимости
def create_an_invertible_matrix(x_matrix, y_matrix, get_inv_matrix=True,):
    
    matrix = np.random.rand(x_matrix,y_matrix)
    while np.linalg.det(matrix) == 0:
        matrix = np.random.rand(x_matrix,y_matrix)
    inv_matrix = np.linalg.inv(matrix)
    if get_inv_matrix:
        return matrix, inv_matrix
    else:
        return matrix

**Обоснование**

Если качество линейной регрессии после умножения матрицы признаков на оборатимую матрицу не ухудшится, то данный алгоритм можно будет использовать для защиты данных.

## Проверка алгоритма

Проверяем качество предсказания на изначальной матрице признаков.

In [7]:
print('Измерение качества первого предсказания R2:',measurement(X,y))

Измерение качества первого предсказания R2: 0.4302010044852067


Формируем обратимую матрицу случайным образом.

In [8]:
A, A_ = create_an_invertible_matrix(X.shape[1],X.shape[1],get_inv_matrix=True)
print(f'случайная матрица А:\n{A}\n\nобратная матрица случайной матрицы A:\n{A_}')

случайная матрица А:
[[0.9193381  0.96802028 0.53746736 0.78155537]
 [0.68380296 0.45832183 0.30313297 0.27789005]
 [0.57531029 0.80188783 0.4663511  0.23607419]
 [0.14493927 0.61762011 0.90146977 0.01923982]]

обратная матрица случайной матрицы A:
[[-1.01977877  3.23909606 -0.41500331 -0.26650008]
 [-0.10714862 -2.70494532  3.61174218 -0.89497503]
 [ 0.18433459  1.36225023 -2.35731483  1.76081287]
 [ 2.48500585 -1.3966349  -2.36417228  0.21108961]]


Умножаем матрицу признаков на обратимую матрицу и получаем второе качество предсказания. 

In [9]:
X_ = X @ A
print('Измерение качества второго предсказания:',measurement(X_,y))

Измерение качества второго предсказания: 0.43020100448518483


**Вывод:**
После умножения матрицы признаков на обратимую матрицу, качество предсказания изменилось лишь в 14-м знаке после запятой что означает, что данный метод можно использовать для шифрования.

**Рекомендация для сотрудников страховой компаниии:**  
Данный метод можно успешно применять для защиты персональных данных клиентов компаниии.